In [1]:
from sklearn.cluster import KMeans
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
tqdm.pandas()

import config

In [2]:
final_df = pd.read_csv(config.final_df)
final_df['preprocessed'] = final_df['preprocessed'].apply(lambda x:eval(x))

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([', '.join(x) for x in final_df.preprocessed])

In [4]:
dictionary = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [5]:
list(dictionary.items())[8690:8700]

[('генит', 11.358044366647004),
 ('гениталии', 5.540933206683799),
 ('генитальный', 9.971750005527113),
 ('генитать', 10.441753634772848),
 ('генрих', 11.358044366647004),
 ('гентсер', 10.441753634772848),
 ('генуэзский', 10.441753634772848),
 ('генуя', 9.971750005527113),
 ('гень', 11.358044366647004),
 ('геогр', 10.664897186087059)]

In [6]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20)

sent = final_df['preprocessed'].values 
w2v_model.build_vocab(sent, progress_per=10000)
w2v_model.train(sent, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(78428120, 159846210)

In [7]:
def text_vector(text):
    vecs = []
    for word in text:
        try:
            vecs.append(w2v_model[word]*dictionary.get(word))
        except KeyError: # not found in w2v_model
            continue
            vecs.append(w2v_model[word]*dictionary.get(word))
        except TypeError: # not found in tf-idf
            continue
    return np.array(vecs).mean(axis=0)

In [8]:
final_df['w2v_weighted'] = final_df['preprocessed'].progress_apply(text_vector)

  0%|          | 0/63018 [00:00<?, ?it/s]/home/marie/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
100%|██████████| 63018/63018 [00:28<00:00, 2206.57it/s]


In [16]:
final_df['w2v_weighted'] = final_df['w2v_weighted'].progress_apply(list)

100%|██████████| 63018/63018 [00:01<00:00, 61192.09it/s]


In [17]:
final_df = final_df[final_df['w2v_weighted'].notna()]

In [18]:
final_df.to_csv(config.final_df, index=False)